In [1]:
!pip install gradio networkx matplotlib
import math
import networkx as nx
import matplotlib.pyplot as plt
import gradio as gr

def find_p_and_m(n):
    for p in range(2, n + 1):
        if all(p % d != 0 for d in range(2, int(math.sqrt(p)) + 1)):
            m = 1
            while p ** m <= n:
                if p ** m == n:
                    return p, m
                m += 1
    return None, None

class PowerDihedralGraph:
    def __init__(self, p, m):
        self.n = p ** m
        if self.n < 3:
            raise ValueError("n harus lebih besar atau sama dengan 3")
        self.G = nx.Graph()

        self.G.add_node('e')
        for i in range(1, self.n):
            self.G.add_node(f'a^{i}')
        for i in range(self.n):
            self.G.add_node(f'a^{i}b')

        for i in range(1, self.n):
            for j in range(1, self.n):
                if i != j and (math.gcd(i, j) > 1):
                    self.G.add_edge(f'a^{i}', f'a^{j}')
        for i in range(self.n):
            for j in range(self.n):
                if i != j and (math.gcd(i, j) > 1):
                    self.G.add_edge(f'a^{i}b', f'a^{j}b')
        for i in range(1, self.n):
            self.G.add_edge('e', f'a^{i}')
            self.G.add_edge('e', f'a^{i}b')

    def zagreb_index_first(self):
        return self.n**2 * (self.n + 1)

    def wiener_index(self):
        return (7 * self.n**2 // 2) - (5 * self.n // 2)

    def gutman_index(self):
        return (1/2) * (self.n**4 + self.n) + (3/2) * (self.n**3 - self.n**2)

    def visualize(self):
        fig, ax = plt.subplots(figsize=(8, 8))
        pos = nx.spring_layout(self.G, seed=42)
        nx.draw(self.G, pos, with_labels=True, node_size=1000, ax=ax)
        ax.set_title(f"Power Dihedral Graph for D_{2*self.n}")
        return fig

    def visualize_indices_single(self):
        zagreb_value = self.zagreb_index_first()
        wiener_value = self.wiener_index()
        gutman_value = self.gutman_index()
        indices = ['Zagreb Index', 'Wiener Index', 'Gutman Index']
        values = [zagreb_value, wiener_value, gutman_value]

        plt.figure(figsize=(8, 5))
        plt.plot(indices, values, marker='o', linestyle='-', color='b')
        plt.xlabel("Index Type")
        plt.ylabel("Value")
        plt.title(f"Index Visualization for n={self.n}")
        plt.grid(True)
        return plt

    def visualize_indices(self):
        n_values = list(range(3, 51))
        zagreb_values = [x**2 * (x + 1) for x in n_values]
        wiener_values = [(7 * x**2 // 2) - (5 * x // 2) for x in n_values]
        gutman_values = [(1/2) * (x**4 + x) + (3/2) * (x**3 - x**2) for x in n_values]

        plt.figure(figsize=(10, 6))
        plt.plot(n_values, zagreb_values, label="Zagreb Index", color='r', marker='o', linestyle='-')
        plt.plot(n_values, wiener_values, label="Wiener Index", color='g', marker='s', linestyle='-')
        plt.plot(n_values, gutman_values, label="Gutman Index", color='b', marker='^', linestyle='-')
        plt.xlabel("n")
        plt.ylabel("Index Value")
        plt.legend()
        plt.title("Visualization of Zagreb, Wiener, and Gutman Indexes Based on n")
        plt.grid(True)
        return plt

def calculate_indices(n):
    try:
        n = int(n)
        if n < 3:
            raise gr.Error("Nilai n harus lebih besar atau sama dengan 3")
        p, m = find_p_and_m(n)
        if p is None or m is None:
            raise gr.Error("Tidak dapat menemukan p dan m untuk n yang diberikan")

        graph = PowerDihedralGraph(p, m)
        return (f"Value of p: {p}\n"
                f"Value of m: {m}\n"
                f"Zagreb Index: {graph.zagreb_index_first()}\n"
                f"Wiener Index: {graph.wiener_index()}\n"
                f"Gutman Index: {graph.gutman_index()}")
    except Exception as e:
        raise gr.Error(str(e))

def visualize_graph(n):
    try:
        n = int(n)
        if n < 3:
            raise gr.Error("Nilai n harus lebih besar atau sama dengan 3")
        p, m = find_p_and_m(n)
        if p is None or m is None:
            raise gr.Error("Tidak dapat menemukan p dan m untuk n yang diberikan")
        return PowerDihedralGraph(p, m).visualize()
    except Exception as e:
        raise gr.Error(str(e))

def visualize_indices(n):
    try:
        n = int(n)
        if n < 3:
            raise gr.Error("Nilai n harus lebih besar atau sama dengan 3")
        p, m = find_p_and_m(n)
        if p is None or m is None:
            raise gr.Error("Tidak dapat menemukan p dan m untuk n yang diberikan")
        return PowerDihedralGraph(p, m).visualize_indices()
    except Exception as e:
        raise gr.Error(str(e))

def visualize_indices_single(n):
    try:
        n = int(n)
        if n < 3:
            raise gr.Error("Nilai n harus lebih besar atau sama dengan 3")
        p, m = find_p_and_m(n)
        if p is None or m is None:
            raise gr.Error("Tidak dapat menemukan p dan m untuk n yang diberikan")
        return PowerDihedralGraph(p, m).visualize_indices_single()
    except Exception as e:
        raise gr.Error(str(e))

with gr.Blocks() as app:
    gr.Markdown("# 📊 Analysis of Indexes in Power Graphs of Dihedral Groups")

    with gr.Row():
        with gr.Column():
            n_input = gr.Number(label="Value of n", precision=0)
            calc_btn = gr.Button("Calculate Indexes")
            viz_btn = gr.Button("Graph Visualization")
            viz_indices_btn = gr.Button("Index Visualization")
            viz_indices_single_btn = gr.Button("Single Index Visualization")

        with gr.Column():
            output_indices = gr.Textbox(label="Computation Result")
            output_plot = gr.Plot(label="Graph Visualization")
            output_indices_plot = gr.Plot(label="Index Visualization")
            output_indices_single_plot = gr.Plot(label="Single Index Visualization")

    calc_btn.click(calculate_indices, [n_input], output_indices)
    viz_btn.click(visualize_graph, [n_input], output_plot)
    viz_indices_btn.click(visualize_indices, [n_input], output_indices_plot)
    viz_indices_single_btn.click(visualize_indices_single, [n_input], output_indices_single_plot)

app.launch()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://069ef69a7494e44ca9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory 